In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor  
from sklearn.tree import DecisionTreeRegressor
from math import radians, cos, sin, asin, sqrt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation


ds1=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/train.csv')
ds2=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/test.csv')
ds10=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/submission.csv')
ds1=pd.DataFrame(ds1)
ds2=pd.DataFrame(ds2)
ds10=pd.DataFrame(ds10)

ds1.drop(['Id', 'Province/State'], axis='columns', inplace=True)

labelencoder_Y = LabelEncoder()
ds1.iloc[:,0] = labelencoder_Y.fit_transform(ds1.iloc[:,0].values)

def single_pt_haversine(lat, lng, degrees=True):
    """
    'Single-point' Haversine: Calculates the great circle distance
    between a point on Earth and the (0, 0) lat-long coordinate
    """
    r = 6371 # Earth's radius (km). Have r = 3956 if you want miles

    # Convert decimal degrees to radians
    if degrees:
        lat, lng = map(radians, [lat, lng])

    # 'Single-point' Haversine formula
    a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
    d = 2 * r * asin(sqrt(a)) 

    return d

ds3 = pd.DataFrame(ds1)
cols2 = [1, 2]
ds4 = ds3[ds3.columns[cols2]]
ds4['harvesine_distance'] = [single_pt_haversine(Lat, Long) for Lat, Long in zip(ds4.Lat, ds4.Long)]

HAVERSINE = ds4["harvesine_distance"]
ds1=ds1.join(HAVERSINE)

ds1.drop(['Lat', 'Long'], axis='columns', inplace=True)
from datetime import datetime
from datetime import timedelta
today = datetime.today()
g = len(ds1["Date"])

for i in range(g):
  ds1["Date"][i] = pd.Timestamp(ds1["Date"][i]).value

ds5 = pd.DataFrame(ds1)
cols3 = [2, 3]
ds6 = ds5[ds5.columns[cols3]]
ds1.drop(['ConfirmedCases', 'Fatalities'], axis='columns', inplace=True)
x_train, x_test, y_train, y_test = train_test_split(ds1, ds6, test_size = 0.25)

X = ds2.iloc[:,0].values
X=pd.DataFrame(X)
ds2.drop(['ForecastId', 'Province/State'], axis='columns', inplace=True)

labelencoder_Y = LabelEncoder()
ds2.iloc[:,0] = labelencoder_Y.fit_transform(ds2.iloc[:,0].values)


def single_pt_haversine(lat, lng, degrees=True):
    """
    'Single-point' Haversine: Calculates the great circle distance
    between a point on Earth and the (0, 0) lat-long coordinate
    """
    r = 6371 # Earth's radius (km). Have r = 3956 if you want miles

    # Convert decimal degrees to radians
    if degrees:
        lat, lng = map(radians, [lat, lng])

    # 'Single-point' Haversine formula
    a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
    d = 2 * r * asin(sqrt(a)) 

    return d

ds7 = pd.DataFrame(ds2)
cols2 = [1, 2]
ds8 = ds7[ds7.columns[cols2]]
ds8['harvesine_distance'] = [single_pt_haversine(Lat, Long) for Lat, Long in zip(ds8.Lat, ds8.Long)]

HAVERSINE = ds8["harvesine_distance"]
ds2=ds2.join(HAVERSINE)

ds2.drop(['Lat', 'Long'], axis='columns', inplace=True)
from datetime import datetime
from datetime import timedelta
today = datetime.today()
g = len(ds2["Date"])

for i in range(g):
  ds2["Date"][i] = pd.Timestamp(ds2["Date"][i]).value


#clf = LinearRegression()
#clf.fit(x_train, y_train)
#clf.score(x_train, y_train)
#clf.predict(x_test)

#tree = DecisionTreeRegressor()
#tree.fit(x_train, y_train)
#tree.score(x_train, y_train)
#tree.predict(x_test)

forest = RandomForestRegressor()
forest.fit(x_train, y_train)
forest.score(x_train, y_train)
forest.predict(x_test)

#import xgboost as xgb
#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)
#xg_reg.fit(x_train,y_train)
#xg_reg.predict(x_test)

#model = Sequential()
#model.add(Dense(6, activation='relu', input_dim=x_train.shape[1]))
#model.add(Dropout(0.5))
#model.add(Dense(6, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(y_train.shape[1], activation='sigmoid'))

#sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='binary_crossentropy',
#              optimizer= 'adam')
#model.fit(x_train, y_train, epochs=50, batch_size=8)


test = ds2
yhat = forest.predict(test)

print('Predicted: %s' % yhat)
yhat = pd.DataFrame(yhat)
ds10.drop(['ForecastId'], axis='columns', inplace=True)
yhat.columns = ds10.columns
X = X.join(yhat)
X.rename(columns = {0:'ForecastId'}, inplace = True)
X.to_csv('submission.csv',index=False)


In [ ]:
x_train.shape[1]